In [2]:
import numpy as np
import pandas as pd



x0=np.array([150, 85, 150, 145, 130, 0])
y0=np.array([140, 85, 155, 50, 150, 0])
q=np.array([243, 236, 220.5, 159, 230, 52])
d=np.zeros((6,6)); a0=np.zeros((6,6)); b0=np.zeros((6,6))
xy0=np.c_[x0,y0]


for i in range(6):
    for j in range(6):
        d[i,j]=np.linalg.norm(xy0[i]-xy0[j])


d[np.where(d==0)]=np.inf
a0=np.arcsin(8./d)*180/np.pi
xy1=x0+1j*y0; xy2=np.exp(1j*q*np.pi/180)


for m in range(6):
    for n in range(6):
        if n!=m:
            b0[m,n]=np.angle((xy2[n]-xy2[m])/(xy1[m]-xy1[n]))
b0=b0*180/np.pi
f=pd.ExcelWriter('./整数规划与非线性规划/Pan6_1.xlsx')  #创建文件对象
pd.DataFrame(a0).to_excel(f,"sheet1",index=None)  #把a0写入Excel文件
pd.DataFrame(b0).to_excel(f,"sheet2",index=None)  #把b0写入表单2
f.save()

In [4]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize
a0=pd.read_excel("./整数规划与非线性规划/Pan6_1.xlsx")    #读入第1个表单
b0=pd.read_excel("./整数规划与非线性规划/Pan6_1.xlsx",1)  #读入第2个表单
a0=a0.values; b0=b0.values  #提取数值
obj=lambda x: np.sum(np.abs(x))
bd=[(-30,30) for i in range(6)]   #决策向量的界限
x0=np.ones((1,6))

cons=[]
for i in range(5):
    for j in range(i+1,6):
        cons.append({'type': 'ineq', 'fun': lambda x: np.abs(b0[i,j]+(x[i]+x[j])/2)-a0[i,j]})

res = minimize(obj, np.ones((1, 6)), constraints=cons, bounds=bd)
print(res)

     fun: 0.7909172985328354
     jac: array([-1., -1., -1., -1.,  1.,  1.])
 message: 'Optimization terminated successfully'
    nfev: 54
     nit: 6
    njev: 6
  status: 0
 success: True
       x: array([2.53262997e-07, 2.53262998e-07, 2.53262998e-07, 2.53262998e-07,
       3.95458143e-01, 3.95458143e-01])


In [5]:
import cvxpy as cp
from numpy import array


c=array([40,90])  #定义目标向量
a=array([[9,7],[-7,-20]])  #定义约束矩阵
b=array([56,-70])  #定义约束条件的右边向量
x=cp.Variable(2,integer=True)  #定义两个整数决策变量
obj=cp.Minimize(c*x)  #构造目标函数
cons=[a*x<=b, x>=0]    #构造约束条件
prob=cp.Problem(obj, cons)  #构建问题模型
prob.solve(solver='GLPK_MI',verbose =True)  #求解问题
print("最优值为:",prob.value)
print("最优解为：\n",x.value)

                                     CVXPY                                     
                                     v1.2.0                                    
(CVXPY) Aug 12 07:08:20 PM: Your problem has 2 variables, 2 constraints, and 0 parameters.
(CVXPY) Aug 12 07:08:20 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Aug 12 07:08:20 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Aug 12 07:08:20 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Aug 12 07:08:20 PM: Compiling problem (target solver=GLPK_MI).
(CVXPY) Aug 12 07:08:20 PM: Reduction chain: Dcp2Cone -> CvxAttr2Constr -> ConeMatrixStuffing 

d:\Anaconda3\lib\site-packages\cvxpy\expressions\expression.py:593: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.
This code path has been hit 1 times so far.

  warnings.warn(msg, UserWarning)
d:\Anaconda3\lib\site-packages\cvxpy\expressions\expression.py:593: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.
This code path has been hit 2 times so far.

  warnings.warn(msg, UserWarning)


In [6]:
import cvxpy as cp
import numpy as np



c=np.array([[4, 8, 7, 15, 12],
            [7, 9, 17, 14, 10],
            [6, 9, 12, 8, 7],
            [6, 7, 14, 6, 10],
            [6, 9, 12, 10, 6]])
x = cp.Variable((5,5),integer=True)
obj = cp.Minimize(cp.sum(cp.multiply(c,x)))
con= [0 <= x, x <= 1, cp.sum(x, axis=0, keepdims=True)==1,\
    cp.sum(x, axis=1, keepdims=True)==1]
prob = cp.Problem(obj, con)
prob.solve(solver='GLPK_MI')
print("最优值为:",prob.value)
print("最优解为：\n",x.value)


最优值为: 34.0
最优解为：
 [[0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]]


In [7]:
import cvxpy as cp
import numpy as np



L=np.array([48.7,52.0,61.3,72.0,48.7,52.0,64.0])
w=np.array([2000,3000,1000,500,4000,2000,1000])
a=np.array([8,7,9,6,6,4,8])
x=cp.Variable((2,7), integer=True)
obj=cp.Maximize(cp.sum(x*L))
con=[cp.sum(x,axis=0,keepdims=True)<=a.reshape(1,7),\
    x*L<=1020, x*w<=40000, cp.sum(x[:,4:]*L[4:])<=302.7, x>=0]
prob = cp.Problem(obj, con)
prob.solve(solver='GLPK_MI',verbose =True)
print("最优值为:",prob.value)
print("最优解为：\n",x.value)

                                     CVXPY                                     
                                     v1.2.0                                    
(CVXPY) Aug 12 08:08:39 PM: Your problem has 14 variables, 5 constraints, and 0 parameters.
(CVXPY) Aug 12 08:08:39 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Aug 12 08:08:39 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Aug 12 08:08:39 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Aug 12 08:08:39 PM: Compiling problem (target solver=GLPK_MI).
(CVXPY) Aug 12 08:08:39 PM: Reduction chain: FlipObjective -> Dcp2Cone -> CvxAttr2Constr -> C

d:\Anaconda3\lib\site-packages\cvxpy\expressions\expression.py:593: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.
This code path has been hit 3 times so far.

  warnings.warn(msg, UserWarning)
d:\Anaconda3\lib\site-packages\cvxpy\expressions\expression.py:593: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.
This code path has been hit 4 times so far.

  warnings.warn(msg, UserWarning)
d:\Anaconda3\lib\site-packages\cvxpy\express

-------------------------------------------------------------------------------
                                    Summary                                    
-------------------------------------------------------------------------------
(CVXPY) Aug 12 08:08:40 PM: Problem status: optimal
(CVXPY) Aug 12 08:08:40 PM: Optimal value: 2.039e+03
(CVXPY) Aug 12 08:08:40 PM: Compilation took 1.445e-02 seconds
(CVXPY) Aug 12 08:08:40 PM: Solver (including time spent in interface) took 9.465e-01 seconds
最优值为: 2039.4
最优解为：
 [[4. 1. 5. 3. 3. 2. 0.]
 [4. 6. 4. 3. 0. 1. 0.]]


In [8]:
import cvxpy as cp
import numpy as np



L=np.array([48.7,52.0,61.3,72.0,48.7,52.0,64.0])
w=np.array([2000,3000,1000,500,4000,2000,1000])
a=np.array([8,7,9,6,6,4,8])
x=cp.Variable((2,7), integer=True)
obj=cp.Maximize(cp.sum(x*w))
con=[cp.sum(x,axis=0,keepdims=True)<=a.reshape(1,7), \
    x*L<=1020, x*w<=40000, cp.sum(x[:,4:]*L[4:])<=302.7,x>=0]
prob = cp.Problem(obj, con)
prob.solve(solver='GLPK_MI',verbose =True)
print("最优值为:",prob.value)
print("最优解为：\n",x.value)

                                     CVXPY                                     
                                     v1.2.0                                    
(CVXPY) Aug 12 08:21:00 PM: Your problem has 14 variables, 5 constraints, and 0 parameters.
(CVXPY) Aug 12 08:21:00 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Aug 12 08:21:00 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Aug 12 08:21:00 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Aug 12 08:21:00 PM: Compiling problem (target solver=GLPK_MI).
(CVXPY) Aug 12 08:21:00 PM: Reduction chain: FlipObjective -> Dcp2Cone -> CvxAttr2Constr -> C

d:\Anaconda3\lib\site-packages\cvxpy\expressions\expression.py:593: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.
This code path has been hit 7 times so far.

  warnings.warn(msg, UserWarning)
d:\Anaconda3\lib\site-packages\cvxpy\expressions\expression.py:593: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.
This code path has been hit 8 times so far.

  warnings.warn(msg, UserWarning)
d:\Anaconda3\lib\site-packages\cvxpy\express

In [9]:
from scipy.optimize import minimize
from numpy import ones


def obj(x):
    x1,x2,x3=x
    return (2+x1)/(1+x2)-3*x1+4*x3


LB=[0.1]*3; UB=[0.9]*3
bound=tuple(zip(LB, UB))   #生成决策向量界限的元组
res=minimize(obj,ones(3),bounds=bound) #第2个参数为初值
print(res.fun,'\n',res.success,'\n',res.x)  #输出最优值、求解状态、最优解


-0.7736842105263159 
 True 
 [0.9 0.9 0.1]


In [10]:
from scipy.optimize import minimize
import numpy as np



c1=np.array([1,1,3,4,2]); c2=np.array([-8,-2,-3,-1,-2])
A=np.array([[1,1,1,1,1],[1,2,2,1,6],
            [2,1,6,0,0],[0,0,1,1,5]])
b=np.array([400,800,200,200])
obj=lambda x: np.dot(-c1,x**2)+np.dot(-c2,x)
cons={'type':'ineq','fun':lambda x:b-A@x}
bd=[(0,99) for i in range(A.shape[1])]
res=minimize(obj,np.ones(5)*90,constraints=cons,bounds=bd)
print(res)  #输出解的信息

     fun: -51629.930000636145
     jac: array([ -93.        , -196.        ,    3.        , -791.        ,
        -78.79980469])
 message: 'Optimization terminated successfully'
    nfev: 36
     nit: 5
    njev: 4
  status: 0
 success: True
       x: array([50.5, 99. ,  0. , 99. , 20.2])


In [11]:
import numpy as np
from cvxopt import matrix,solvers



n=3; P=matrix(0.,(n,n))
P[::n+1]=[3,2,1.7]; q=matrix([3,-8.2,-1.95])
A=matrix([[1.,0,1],[-1,2,0],[0,1,2]]).T
b=matrix([2.,2,3])
Aeq=matrix(1.,(1,n)); beq=matrix(3.)
s=solvers.qp(P,q,A,b,Aeq,beq)
print("最优解为：",s['x'])
print("最优值为：",s['primal objective'])

     pcost       dcost       gap    pres   dres
 0: -1.3148e+01 -4.4315e+00  1e+01  1e+00  9e-01
 1: -6.4674e+00 -7.5675e+00  1e+00  1e-16  2e-16
 2: -7.1538e+00 -7.1854e+00  3e-02  0e+00  7e-16
 3: -7.1758e+00 -7.1761e+00  3e-04  1e-16  1e-15
 4: -7.1760e+00 -7.1760e+00  3e-06  1e-16  7e-16
Optimal solution found.
最优解为： [ 8.00e-01]
[ 1.40e+00]
[ 8.00e-01]

最优值为： -7.175997768777275


In [12]:
import cvxpy as cp
import numpy as np



c1=np.array([1, 1, 3, 4, 2])
c2=np.array([-8, -2, -3, -1, -2])
a=np.array([[1, 1, 1, 1, 1], [1, 2, 2, 1, 6], [2, 1, 6, 0, 0], [0, 0, 1, 1, 5]])
b=np.array([400, 800, 200, 200])
x=cp.Variable(5,integer=True)
obj=cp.Minimize(c1*x**2+c2*x)
con=[0<=x, x<=99, a*x<=b]
prob = cp.Problem(obj, con)
prob.solve()
print("最优值为:",prob.value)
print("最优解为：\n",x.value)

d:\Anaconda3\lib\site-packages\cvxpy\expressions\expression.py:593: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.
This code path has been hit 11 times so far.

  warnings.warn(msg, UserWarning)
d:\Anaconda3\lib\site-packages\cvxpy\expressions\expression.py:593: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.
This code path has been hit 12 times so far.

  warnings.warn(msg, UserWarning)
d:\Anaconda3\lib\site-packages\cvxpy\expre

SolverError: Either candidate conic solvers (['GLPK_MI']) do not support the cones output by the problem (SOC, NonNeg), or there are not enough constraints in the problem.